# Clustering text

See also https://towardsdatascience.com/nmf-a-visual-explainer-and-python-implementation-7ecdd73491f8


This notebook uses a collection of clinical case reports to cluster words by topics using the NMF method. To cluster text we need to preprocess the text first with regular natural language processing cleaning steps such as remove punctuations, stopwords, or other unwanted text. we lower the text and use the lemma to reduce variation of words. This is all done in part A. 

Next we need to prepare the text in a document term matrix so that NMF can perform the calculations. The Document-Term Matrix (DTM) represents the frequency of words (or terms) in a collection of documents. Each row in the matrix represents a document, and each column represents a word in the vocabulary. The value in each cell represents the frequency of the corresponding word in the corresponding document. This is done in part B

Lastly we run the clustering algorithm and visualize the outcome. This is done in part C


## The data 
A collection of 200 clinical case report documents in plain text format are used. The documents are named using PubMed document IDs, and have been edited to include only clinical case report details. The dataset is called "MACCROBAT2020" and is the second release of this dataset, with improvements made to the consistency and format of annotations
https://figshare.com/articles/dataset/MACCROBAT2018/9764942


## Portfolio assignment

You can use this assignment to fill your portfolio.
Read, execute and analyse the code in the notebook. Then *choose one* of the assignments a), b) or c). 

a) read the article Clinical Documents Clustering Based on Medication/Symptom Names Using Multi-View Nonnegative Matrix Factorization. you can find the article <a href = 'https://pubmed.ncbi.nlm.nih.gov/26011887/'> here</a>. Explain the similarities of this notebook and the article. Explain in your own words what need to be added to this notebook to reproduce the article. There is no need to code the solution.

b) Improve the outcome improving the data preprocessing and the hyper parameter configurations. Explain your choices. Your solution should be a coded solution with comments. Are there any other weighting solutions next to TF-IDF?

c) Provide a text clustering solution with your own data of interest, for instance text you work with in your project. 

Mind you that you are not allowed to copy code solutions without referencing. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import re
import string
import glob
import pandas as pd
from pathlib import Path

# Part A: get and clean the text

## Get the data

In [ ]:
import glob
import pandas as pd
from pathlib import Path

#create empty dataframe
df = pd.DataFrame(columns=['docid','text'])

# get all files ending with .txt
docs = [x for x in glob.glob(r"C:\zshahpouri\Machin_Learning\Data\*.txt")]

#fill dataframe
for doc in docs:
    txt = Path(doc).read_text(encoding='utf-8', errors='ignore')
    docid = Path(doc).stem  # Extract the filename without the extension
    df.loc[len(df.index)] = [docid, txt]

#set index docid       
df = df.set_index('docid')


In [ ]:
df.head()

## Cleaning the text

The code below defines a function called clean_text that takes a text parameter and returns a cleaned version of it. The function first converts the text to lowercase using the `lower()` method. It then removes any text enclosed in square brackets using **regular expressions** and the re.sub() method. Next, it removes any **punctuation** from the text using the `string.punctuation` module and `re.escape()` and `re.sub()` methods. It then removes any words that **contain numbers** using `re.sub()` and a regular expression. Finally, it removes any **read errors** (represented by the � character) using `re.sub()`.

The code then defines a lambda function called cleaned that takes a single parameter `x` and applies the `clean_text` function to it. This lambda function can be used to clean text data in a Pandas dataframe or any other data structure that can take a lambda function

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, 
    remove punctuation, remove read errors,
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('�', ' ', text)
    return text

cleaned = lambda x: clean_text(x)

The function 'nouns' cleans a text and extracts only the nouns from it. It first defines a lambda function called 'is_noun' that checks if a given word is a noun, based on its part of speech (POS) tag. The input text is then tokenized using the `word_tokenize` function from the **nltk** module, which breaks down the text into smaller units called tokens. Tokenization is an important pre-processing step in NLP that helps standardize and prepare text data for further analysis.

The function creates a `WordNetLemmatizer` object from the nltk module to lemmatize each word, which is the process of converting a word to its base or dictionary form. Lemmatization helps to reduce the dimensionality of text data and improve the accuracy of the analysis, especially in cases where words have different inflected forms but share the same root.

The function uses a list comprehension to lemmatize each word in the tokenized text if it is a noun, and stores the resulting list of lemmatized nouns in the 'all_nouns' variable. Finally, the function returns a string containing the joined list of lemmatized nouns.

Note that this function requires the 'nltk' module to be imported and assumes that the 'pos_tag' function from the 'nltk' module is available.

In [ ]:
# Noun extract and lemmatize function
def nouns(text):
    '''Given a string of text, tokenize the text 
    and pull out only the nouns.'''
    # create mask to isolate words that are nouns
    is_noun = lambda pos: pos[:2] == 'NN'
    # store function to split string of words 
    # into a list of words (tokens)
    tokenized = word_tokenize(text)
    # store function to lemmatize each word
    wordnet_lemmatizer = WordNetLemmatizer()
    # use list comprehension to lemmatize all words 
    # and create a list of all nouns
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
    for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    
    #return string of joined list of nouns
    return ' '.join(all_nouns)


In [ ]:
# Clean Text
df["text"] = df["text"].apply(cleaned)
data_nouns = pd.DataFrame(df["text"].apply(nouns))
# Visually Inspect
data_nouns.head()

In [ ]:
data_nouns.text[0]

# Part B: The Document-Term Matrix (DTM)

To perform analyses we need to create a Document-Term Maxtrix. 
The Document-Term Matrix (DTM) represents the frequency of words (or terms) in a collection of documents. Each row in the matrix represents a document, and each column represents a word in the vocabulary. The value in each cell represents the frequency of the corresponding word in the corresponding document.

In specific case below, the DTM has been created using only the nouns extracted from the original text, and has been transformed using the TF-IDF (Term Frequency-Inverse Document Frequency) weighting scheme. The TF-IDF scheme assigns weights to words based on how often they appear in a document relative to how often they appear in the entire corpus. Words that appear frequently in a document but infrequently in the corpus are given higher weights, as they are considered to be more important for distinguishing that document from others in the corpus. This weighting scheme is commonly used in text mining and information retrieval to identify key terms or topics in a collection of documents.

The resulting DTM can be used for various purposes such as topic modeling, clustering, classification, and visualization of text data.

In [ ]:
# Create a document-term matrix with only nouns
# Store TF-IDF Vectorizer
tv_noun = TfidfVectorizer(stop_words='english', ngram_range = (1,1), max_df = .8, min_df = .01)
# Fit and Transform speech noun text to a TF-IDF Doc-Term Matrix
data_tv_noun = tv_noun.fit_transform(data_nouns.text)
# Create data-frame of Doc-Term Matrix with nouns as column names
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names_out())
data_dtm_noun.index = df.index
# Visually inspect Document Term Matrix
data_dtm_noun.head()

# Part C: Run the NMF

This code below performs Non-negative Matrix Factorization (NMF) on the Document Term Matrix (DTM) with only nouns. NMF is a dimensionality reduction technique that decomposes a matrix into two matrices: a document-topic matrix (W) and a topic-term matrix (H).

The code first creates an NMF model with 5 topics. The model is then fit to the DTM with the fit_transform() method, which returns the document-topic matrix (W).

The `display_topics()` function is called to extract the top words from the topic-term matrix (H) using the feature names from the TF-IDF vectorizer. 

The resulting output will show the top n words for each of the n topics learned by the NMF model.

In [ ]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    '''Given an NMF model, feature_names, and number of top words, print 
       topic number and its top feature names, up to specified number of top words.'''
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [ ]:
nmf_model = NMF(5)
# Learn an NMF model for given Document Term Matrix 'V' 
# Extract the document-topic matrix 'W'
doc_topic = nmf_model.fit_transform(data_dtm_noun)
# Extract top words from the topic-term matrix 'H' 
display_topics(nmf_model, tv_noun.get_feature_names_out(), 10)

For inspiration of visualizations see https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

---------

## Similarities:

This script and the article both focus on the process of extracting meaningful information from text data. Specifically, they both involve the application of Nonnegative Matrix Factorization (NMF) for topic modeling and clustering purposes.

## Differences:

The script is designed to work with a general texts. It just processes the texts by cleaning them, extracting and lemmatizing nouns, and then applying NMF for topic modeling. It does not differentiate between types of nouns or consider different "views" of the data.
On the other hand, the article outlines a system specifically for clinical documents. It focuses on extracting medication names and symptom names from clinical notes, which can be seen as a form of feature extraction. The extracted features are then used for clustering using NMF and multi-view NMF.


In order to reproduce the approach from the article using the notebook, the following steps should be considered:

### Domain-Specific Text Processing:
Rather than just extracting and lemmatizing all nouns, the notebook needs to incorporate a mechanism for specifically extracting medication and symptom names. This could involve the use of domain-specific ontologies, regular expressions, or even machine learning-based named entity recognition (NER) techniques.

### Multi-view NMF Implementation:
Our script currently uses a basic NMF implementation, but the article uses multi-view NMF. You would need to add code to implement or call a multi-view NMF algorithm.

### Symptom/Medication-Based Clustering:
The script currently does not perform any clustering, while the article specifically focuses on clustering based on the extracted features (medication and symptom names). Thus, clustering methods should be added to the notebook.

### Evaluation and Comparison:
The article presents a comparison of the performances of NMF and multi-view NMF on clinical documents clustering. 


----


Fatemeh and I collaborated on this assignment.